In [1]:
import cv2
import numpy as np

from tensorflow import keras

In [2]:
my_model = keras.models.load_model("my_model")
my_model.load_weights("weights.h5")

In [3]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img

In [4]:
def getCalssName(classNo):
    if classNo == 0:
        return "Speed Limit 20 km/h"
    elif classNo == 1:
        return "Speed Limit 30 km/h"
    elif classNo == 2:
        return "Speed Limit 50 km/h"
    elif classNo == 3:
        return "Speed Limit 60 km/h"
    elif classNo == 4:
        return "Speed Limit 70 km/h"
    elif classNo == 5:
        return "Speed Limit 80 km/h"
    elif classNo == 6:
        return "End of Speed Limit 88 km/h"
    elif classNo == 7:
        return "Speed Limit 100 km/h"
    elif classNo == 8:
        return "Speed Limit 120 km/h"
    elif classNo == 9:
        return "No passing"
    elif classNo == 18:
        return "No passing for vechiles over 3.5 metric tons"
    elif classNo == 11:
        return "Right-of-way at the next intersection"
    elif classNo == 12:
        return "Priority road"
    elif classNo == 13:
        return "Yield"
    elif classNo == 14:
        return "Stop"
    elif classNo == 15:
        return "No vechiles"
    elif classNo == 16:
        return "Vechiles over 3.5 metric tons prohibited"
    elif classNo == 17:
        return "No entry"
    elif classNo == 18:
        return "General caution"
    elif classNo == 19:
        return "Dangerous curve to the left"
    elif classNo == 20:
        return "Dangerous curve to the right"
    elif classNo == 21:
        return "Double curve"
    elif classNo == 22:
        return "Bumpy road"
    elif classNo == 23:
        return "Slippery road"
    elif classNo == 24:
        return "Road narrows on the right"
    elif classNo == 25:
        return "Road work"
    elif classNo == 26:
        return "Traffic signals"
    elif classNo == 27:
        return "Pedestrians"
    elif classNo == 28:
        return "Children crossing"
    elif classNo == 29:
        return "Bicycles crossing"
    elif classNo == 30:
        return "Beware of ice/snow"
    elif classNo == 31:
        return "Wild animals crossing"
    elif classNo == 32:
        return "End of all speed and passing limits"
    elif classNo == 33:
        return "Turn right ahead"
    elif classNo == 34:
        return "Turn left ahead"
    elif classNo == 35:
        return "Ahead only"
    elif classNo == 36:
        return "Go straight or right"
    elif classNo == 37:
        return "Go straight or left"
    elif classNo == 38:
        return "Keep right"
    elif classNo == 39:
        return "Keep left"
    elif classNo == 40:
        return "Roundabout mandatory"
    elif classNo == 41:
        return "End of no passing"
    elif classNo == 42:
        return "End of no passing by vechiles over 3.5 metric tons"

In [23]:
# Define the coordinates for the top-left and bottom-right corners of the box
x1, y1 = 100, 100  # Top-left corner
x2, y2 = 320, 320  # Bottom-right corner

# Draw a red rectangle around the defined area
color = (0, 0, 255)  # BGR color (red in this case)
thickness = 2

In [24]:
frameWidth = 640          # CAMERA RESOLUTION
frameHeight = 480
brightness = 180
threshold = 0.85          # PROBABLITY THRESHOLD
font = cv2.FONT_HERSHEY_SIMPLEX

In [25]:
# SETUP THE VIDEO CAMERA
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, brightness)

while True:
    # READ IMAGE
    success, imgOrignal = cap.read()
    # PROCESS IMAGE
    cv2.rectangle(imgOrignal, (x1, y1), (x2, y2), color, thickness)
    img = imgOrignal[y1:y2, x1:x2]
    img = np.asarray(img)
    img = cv2.resize(img, (32, 32))
    img = preprocessing(img)

    img = img.reshape(1, 32, 32, 1)
    cv2.putText(imgOrignal, "TRAFFIC SIGN: ", (20, 35),
                font, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(imgOrignal, "PROBABILITY: ", (20, 75),
                font, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
    # PREDICT IMAGE
    predictions = my_model.predict(img, verbose=0)
    classIndex = np.argmax(predictions)
    probabilityValue = np.amax(predictions)
    prob = round(probabilityValue * 100, 2)

    if probabilityValue > threshold:
        # print(getCalssName(classIndex))
        cv2.putText(imgOrignal, str(classIndex)+" "+str(getCalssName(classIndex)),
                    (188, 35), font, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(imgOrignal, str(prob)+"%", (188, 75),
                    font, 0.75, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow("cameraFeed", imgOrignal)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()